In [ ]:
# flares

In [ ]:
import os, shutil

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

 Amamas_visa_application     Home_purchase		 Paper_reviews	 turning_off_bitlocker.docx
 Anime_images		     Industry_training		 Recipes	'Untitled document.gdoc'
 catdog_with_indices.ipynb   ML_project			 Research	 Vava_maulo_art_project
'Colab Notebooks'	     NGIMS_gravity_waves_codes	 Research_data
 GEDI.gdoc		     Paper_2			 Tax_2023


In [ ]:
import os

# Path to the training data
path_to_train_data = "/content/drive/My Drive/ML_project/all_cat_dog_data"

# Lists to store the paths of cat and dog images
cat_image_paths = []
dog_image_paths = []

for filename in os.listdir(path_to_train_data):
    if 'cat' in filename:
        cat_image_paths.append(os.path.join(path_to_train_data, filename))
    elif 'dog' in filename:
        dog_image_paths.append(os.path.join(path_to_train_data, filename))

print(f"Number of cat images: {len(cat_image_paths)}")
print(f"Number of dog images: {len(dog_image_paths)}")


Number of cat images: 12500
Number of dog images: 12500


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
model_resnet18 = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
model_resnet34 = torch.hub.load('pytorch/vision', 'resnet34', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s]
Using cache found in /root/.cache/torch/hub/pytorch_vision_main
/usr/local/lib/python3.10/dist-

In [ ]:
# Freeze all params except the BatchNorm layers, as here they are trained to the
# mean and standard deviation of ImageNet and we may lose some signal
for name, param in model_resnet18.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

for name, param in model_resnet34.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

In [ ]:
# Replace the classifier
num_classes = 2

model_resnet18.fc = nn.Sequential(nn.Linear(model_resnet18.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

model_resnet34.fc = nn.Sequential(nn.Linear(model_resnet34.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=5, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets)
            valid_loss += loss.data.item() * inputs.size(0)

            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, accuracy = {:.4f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [ ]:
import random
from torch.utils.data import Dataset, DataLoader, random_split

# Custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


paths_class1 = cat_image_paths[:80]
paths_class2 = dog_image_paths[:20]
split_fractions = [0.8, 0.1, 0.1]

def get_dataloaders(paths_class1, paths_class2, split_fractions):
  # split the paths list into subsets for class 1 and class 2
  paths_class1_split = [list(subset) for subset in random_split(paths_class1, split_fractions)]
  paths_class2_split = [list(subset) for subset in random_split(paths_class2, split_fractions)]

  # defining inputs to the DataLoader function
  img_dimensions = 224
  img_transforms = transforms.Compose([
      transforms.Resize((img_dimensions, img_dimensions)),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
      ])
  batch_size = 32
  num_workers = 2

  # function to get dataloader from class subsets
  def get_dataloader(class1_subset, class2_subset):
    all_subset = class1_subset + class2_subset
    class1_subset_labels = len(class1_subset)*[0]
    class2_subset_labels = len(class2_subset)*[1]
    all_subset_labels = class1_subset_labels + class2_subset_labels
    subset_dataset = CustomImageDataset(all_subset, all_subset_labels, transform=img_transforms)
    subset_dataloader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    return subset_dataloader

  data_loaders = [get_dataloader(class1_subset, class2_subset) for class1_subset, class2_subset in zip(paths_class1_split, paths_class2_split)]

  return data_loaders

train_data_loader, validation_data_loader, test_data_loader = get_dataloaders(paths_class1, paths_class2, split_fractions)
train_data_loader, validation_data_loader, test_data_loader

(<torch.utils.data.dataloader.DataLoader at 0x7c98a96a2830>,
 <torch.utils.data.dataloader.DataLoader at 0x7c98a96a3730>)

In [ ]:
print(f'Num training images: {len(train_data_loader.dataset)}')
print(f'Num validation images: {len(validation_data_loader.dataset)}')
print(f'Num test images: {len(test_data_loader.dataset)}')

Num training images: 80
Num validation images: 10
Num test images: 10


## Train and test the models

In [ ]:
def test_model(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_data_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('correct: {:d}  total: {:d}'.format(correct, total))
    print('accuracy = {:f}'.format(correct / total))

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
model_resnet18.to(device)
optimizer = optim.Adam(model_resnet18.parameters(), lr=0.001)
train(model_resnet18, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, validation_data_loader, epochs=2, device=device)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch: 0, Training Loss: 1.7352, Validation Loss: 0.5157, accuracy = 0.9178
Epoch: 1, Training Loss: 0.3863, Validation Loss: 0.1752, accuracy = 0.9711


In [ ]:
test_model(model_resnet18)

correct: 442  total: 450
accuracy = 0.982222


In [ ]:
model_resnet34.to(device)
optimizer = optim.Adam(model_resnet34.parameters(), lr=0.001)
train(model_resnet34, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, validation_data_loader, epochs=2, device=device)

Epoch: 0, Training Loss: 0.1805, Validation Loss: 0.0275, accuracy = 0.9889
Epoch: 1, Training Loss: 0.0398, Validation Loss: 0.0278, accuracy = 0.9911


In [ ]:
test_model(model_resnet34)


correct: 445  total: 450
accuracy = 0.988889


## Make some predictions


In [ ]:
import os
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

def make_prediction(model, filename):
    labels, _ = find_classes('/content/drive/My Drive/ML_project/dogs-vs-cats/test')
    img = Image.open(filename)
    img = img_test_transforms(img)
    img = img.unsqueeze(0)
    prediction = model(img.to(device))
    prediction = prediction.argmax()
    print(labels[prediction])

# make_prediction(model_resnet34, '/content/drive/My Drive/ML_project/dogs-vs-cats/test/dogs/dog.1146.jpg')
# make_prediction(model_resnet34, '/content/drive/My Drive/ML_project/dogs-vs-cats/test/cats/cat.1226.jpg')

In [ ]:
torch.save(model_resnet18.state_dict(), "./model_resnet18.pth")
torch.save(model_resnet34.state_dict(), "./model_resnet34.pth")


## Load the models from disk and test with an ensemble

In [ ]:
# Remember that you must call model.eval() to set dropout and batch normalization layers to
# evaluation mode before running inference. Failing to do this will yield inconsistent inference result

resnet18 = torch.hub.load('pytorch/vision', 'resnet18')
resnet18.fc = nn.Sequential(nn.Linear(resnet18.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))
resnet18.load_state_dict(torch.load('./model_resnet18.pth'))
resnet18.eval()

resnet34 = torch.hub.load('pytorch/vision', 'resnet34')
resnet34.fc = nn.Sequential(nn.Linear(resnet34.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))
resnet34.load_state_dict(torch.load('./model_resnet34.pth'))
resnet34.eval()

print("done")

Using cache found in /root/.cache/torch/hub/pytorch_vision_main


FileNotFoundError: [Errno 2] No such file or directory: './model_resnet18.pth'

In [ ]:
# Test against the average of each prediction from the two models
models_ensemble = [resnet18.to(device), resnet34.to(device)]
correct = 0
total = 0
with torch.no_grad():
    for data in test_data_loader:
        images, labels = data[0].to(device), data[1].to(device)
        predictions = [i(images).data for i in models_ensemble]
        avg_predictions = torch.mean(torch.stack(predictions), dim=0)
        _, predicted = torch.max(avg_predictions, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('accuracy = {:f}'.format(correct / total))
print('correct: {:d}  total: {:d}'.format(correct, total))

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


accuracy = 0.984444
correct: 443  total: 450


In [ ]:
# Assuming your model and data are on the same device (e.g., 'cuda' or 'cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet34.to(device)

# Example usage
make_prediction(resnet34, '/content/drive/My Drive/ML_project/dogs-vs-cats/test/dogs/dog.1146.jpg')
make_prediction(resnet34, '/content/drive/My Drive/ML_project/dogs-vs-cats/test/cats/cat.1226.jpg')

dogs
cats
